The tricount key is the unique identifier of a tricount:

In [1]:
// https://tricount.com/tMjbqgwJxaikhUbkNz
let tricountKey = "tMjbqgwJxaikhUbkNz"

Tricounts are public (they are not protected by a secret password),
but apps must identify themselves before they can use the tricount API.

To authenticate, an app must have a unique random id and a unique RSA public key:

In [2]:
open System.Security.Cryptography

// Generate a new app id (normally generated when the tricount app is installed)
let appInstallationId = Guid.NewGuid().ToString()

// Generate a new public key (no idea what the server does with it but it has to be unique)
let rsaPublicKeyPem = RSA.Create(2048).ExportRSAPublicKeyPem()

Some http headers are also required for each requests:

In [3]:
open System.Net.Http

// Setup an http client with the required headers
let httpClient = new HttpClient()
httpClient.BaseAddress <- Uri("https://api.tricount.bunq.com")
// The User-Agent might not be required, but let's try to avoid detection a little...
httpClient.DefaultRequestHeaders.TryAddWithoutValidation("User-Agent", "com.bunq.tricount.android:RELEASE:7.0.7:3174:ANDROID:13:C")
httpClient.DefaultRequestHeaders.Add("app-id", appInstallationId)
httpClient.DefaultRequestHeaders.Add("X-Bunq-Client-Request-Id", "049bfcdf-6ae4-4cee-af7b-45da31ea85d0") // This is a constant hardcoded in the app

An authentication request can then be made with the information defined above:

In [4]:
open System.Net.Http.Json

let authRequest = new HttpRequestMessage(HttpMethod.Post, "v1/session-registry-installation")
authRequest.Content <- JsonContent.Create(
    {|
        app_installation_uuid = appInstallationId
        client_public_key = rsaPublicKeyPem
        device_description = "Android"
    |})

let authResponse = httpClient.Send(authRequest)
let authResponseBody = authResponse.Content.ReadAsStringAsync().Result

authResponse, authResponseBody

(StatusCode: 200, ReasonPhrase: 'OK', Version: 1.1, Content: System.Net.Http.HttpConnectionResponseContent, Headers:\r\n{\r\n Date: Sat, 01 Mar 2025 14:35:46 GMT\r\n Transfer-Encoding: chunked\r\n Connection: keep-alive\r\n Server: Apache\r\n Access-Control-Allow-Origin: *\r\n x-bunq-client-re... Item1 StatusCode: 200, ReasonPhrase: 'OK', Version: 1.1, Content: System.Net.Http.HttpConnectionResponseContent, Headers:\r\n{\r\n Date: Sat, 01 Mar 2025 14:35:46 GMT\r\n Transfer-Encoding: chunked\r\n Connection: keep-alive\r\n Server: Apache\r\n Access-Control-Allow-Origin: *\r\n x-bunq-client-req... Version 1.1 Major 1 Minor 1 Build -1 Revision -1 MajorRevision -1 MinorRevision -1 Content System.Net.Http.HttpConnectionResponseContent Headers [ [Content-Type, System.String[]] ] Allow [ ] Count 0 IsReadOnly False (values) [ ] ContentDisposition <null> ContentEncoding [ ] Count 0 IsReadOnly False (values) [ ] ContentLanguage [ ] Count 0 IsReadOnly False (values) [ ] ContentLength 2523 ContentLocation <null> ContentMD5 <null> ContentRange <null> ContentType application/json CharSet <null> Parameters [ ] Count 0 IsReadOnly False (values) (empty) MediaType application/json Expires <null> LastModified <null> NonValidated [ [Content-Type, application/json], [Content-Length, 2523] ] Count 2 (values) index value 0 [Content-Type, application/json] Key Content-Type Value [ application/json ] 1 [Content-Length, 2523] Key Content-Length Value [ 2523 ] (values) index value 0 [Content-Type, System.String[]] Key Content-Type Value [ application/json ] 1 [Content-Length, System.String[]] Key Content-Length Value [ 2523 ] StatusCode OK ReasonPhrase OK Headers [ [Date, System.String[]], [Transfer-Encoding, System.String[]], [Connection, System.String[]], [Server, System.String[]], [Access-Control-Allow-Origin, System.String[]], [x-bunq-client-request-id, System.String[]], [x-bunq-client-response-id, System.String[]], [x-bunq-server-signature, System.String[]], [Strict-Transport-Security, System.String[]], [Vary, System.String[]], [X-Content-Type-Options, System.String[]], [X-Frame-Options, System.String[]] ] AcceptRanges [ ] Count 0 IsReadOnly False (values) [ ] Age <null> ETag <null> Location <null> ProxyAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) RetryAfter <null> Server [ Apache ] Count 1 IsReadOnly False (values) index value 0 Apache Product Apache Name Apache Version <null> Comment <null> Vary [ Accept-Encoding ] Count 1 IsReadOnly False (values) [ Accept-Encoding ] WwwAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) CacheControl <null> Connection [ keep-alive ] Count 1 IsReadOnly False (values) [ keep-alive ] ConnectionClose <null> Date 2025-03-01 14:35:46Z Pragma [ ] Count 0 IsReadOnly False (values) (empty) Trailer [ ] Count 0 IsReadOnly False (values) [ ] TransferEncoding [ chunked ] Count 1 IsReadOnly False (values) index value 0 chunked Value chunked Parameters [ ] Count 0 IsReadOnly False (values) (empty) TransferEncodingChunked True Upgrade [ ] Count 0 IsReadOnly False (values) (empty) Via [ ] Count 0 IsReadOnly False (values) (empty) Warning [ ] Count 0 IsReadOnly False (values) (empty) NonValidated [ [Date, Sat, 01 Mar 2025 14:35:46 GMT], [Transfer-Encoding, chunked], [Connection, keep-alive], [Server, Apache], [Access-Control-Allow-Origin, *], [x-bunq-client-request-id, 049bfcdf-6ae4-4cee-af7b-45da31ea85d0], [x-bunq-client-response-id, aacb65fd-44fe-4c89-94fb-e7ebf97ddd8a], [x-bunq-server-signature, smEqpt0KmQFLd0nst+4obwzzRQzcJaN8RPAlrf0Vw7ptoliFiQK5z0xEhGb5ow7aWPUAV6qimX/4HIWNlB65Ka+nLKELRPoUShr+VjrCwR1o+SjoNMvNsP6RZ77NSawUnYmvMeNoEjgdQ8vSZNYCdGXV2VtLuuK1MN8ZbDr8AOrBYFEfXIh9Ii8nuAv8iavYmSL8gMC5HGfgkbtO+rjF/guihq+yx7uYXxnMWqcftpXEh907TsCOPrXx18yUPjqox1KIImxhn48d+I9wYx4o4dWtgqOpHzYgZScJIRPOB8e59S3RIlwPVI4aR1K/1E6mUwPLfurhyAN+KAbpn6PCzg==], [Strict-Transport-Security, max-age=31536000], [Vary, Accept-Encoding], [X-Content-Type-Options, nosniff], [X-Frame-Options, sameorigin] ] Count 12 (values) index value 0 [Da

The authentication response contains a lot of information.  
The only required info are:
- the auth token to be used in subsequent requests
- the assigned user id that is needed to call some endpoints

In [5]:
open System.Text.Json

/// Json helper to match the single-property objects inside the api Response array:
let getObjFromKey (propertyName: string) (input: JsonElement) =
    match input.TryGetProperty(propertyName) with
    | false, _ -> None
    | true, x -> Some x

let session =
    let authResponseJson = JsonDocument.Parse(authResponseBody).RootElement
    // The `Response` property contains an array of objects.
    // Each one of these objects contain a single property, and the value of this property is itself an object.
    // This object is the one containing the useful data.
    let responseItems = authResponseJson.GetProperty("Response").EnumerateArray()
    let tokenObj = responseItems |> Seq.pick (getObjFromKey "Token")
    let userObj = responseItems |> Seq.pick (getObjFromKey "UserPerson")
    let authToken = tokenObj.GetProperty("token").GetString()
    let userId = userObj.GetProperty("id").GetInt32()
    {| AuthToken = authToken; UserId = userId |}

// Authenticate subsequent requests:
httpClient.DefaultRequestHeaders.Add("X-Bunq-Client-Authentication", session.AuthToken)

session

AuthToken,6075f67b4b926a05502a29d93a3f1bc9abe484caa6996030f4cf95bcb287ab14
UserId,50910281


Once we have an authentication token and a user id, we can use the API to query all the information of our tricount:

In [6]:
let tricountJson = httpClient.GetStringAsync($"/v1/user/{session.UserId}/registry?public_identifier_token={tricountKey}").Result
let formattedTricountJson = JsonSerializer.Serialize(JsonDocument.Parse(tricountJson).RootElement, JsonSerializerOptions(WriteIndented=true))
formattedTricountJson

{
  "Response": [
    {
      "Registry": {
        "id": 47790603,
        "created": "2024-02-05 21:03:19.638272",
        "updated": "2024-02-05 21:03:19.704880",
        "uuid": "4f1c50aa-7cc7-466d-9547-04121820802e",
        "currency": "USD",
        "emoji": null,
        "title": "City trip",
        "description": "This is a sample tricount",
        "category": "OTHER",
        "status": "READ_WRITE",
        "membership_uuid_active": null,
        "memberships": [
          {
            "RegistryMembershipNonUser": {
              "id": 199300519,
              "created": "2024-02-05 21:03:19.653411",
              "updated": "2024-02-05 21:03:19.729255",
              "uuid": "e7631a4a-5926-47c5-a2bf-a3097a7b874c",
              "alias": {
                "display_name": "Julia",
                "pointer": {
                  "type": "UUID",
                  "value": "c511809d-32b8-410c-9efb-b8fe2b0f81a4",
                  "name": "Julia"
                }
              

Now that we have the raw tricount data, let's parse it into something meaningful, then format it to an html document.

First, let's define the parser:

In [7]:
#r "nuget: FSharp.Data"

open FSharp.Data

type TriCountJson =
    JsonProvider<
        SampleIsList=true,
        Sample=const(__SOURCE_DIRECTORY__ + "/samples.json")>

type ResultRow = {
    WhoPaid: string
    HowMuch: decimal
    Currency: string
    Description: string
    When: DateTime
    Involved: string
    HowMuchForMe: decimal
}

type ParsedTricount = {
    Title: string
    Memberships: (string * {| Balance: decimal; Currency: string |}) array
    Rows: ResultRow array
    Total: decimal
    TotalForMe: decimal
}

Installed Packages FSharp.Data, 6.4.1

Then let's define the functions to parse and format this json:

In [8]:
/// Parse a tricount json to extract a user-friendly formatted table from it.
/// `myName` is the name of the tricount member for which to compute the "how much for me" detail.
let parseTricountJson myName json =
    let json = TriCountJson.Parse(json)

    let registry = json.Response |> Array.exactlyOne |> (fun x -> x.Registry)

    // Definition of the tricount members and their balances:
    let memberships =
        registry.Memberships
        |> Array.map (fun x ->
            let name = x.RegistryMembershipNonUser.Alias.DisplayName
            let ownedEntries =
                registry.AllRegistryEntry
                |> Array.where (fun x -> x.RegistryEntry.MembershipOwned.RegistryMembershipNonUser.Alias.DisplayName = name)
            let allocations =
                registry.AllRegistryEntry
                |> Array.collect _.RegistryEntry.Allocations
                |> Array.where (fun x -> x.Membership.RegistryMembershipNonUser.Alias.DisplayName = name)

            let sumOwnedEntries = ownedEntries |> Array.sumBy _.RegistryEntry.Amount.Value
            let ownedEntriesCurrencies = ownedEntries |> Array.map _.RegistryEntry.Amount.Currency

            let sumAllocations = allocations |> Array.sumBy _.Amount.Value
            let allocationsCurrencies = allocations |> Array.map _.Amount.Currency

            let balance = sumAllocations - sumOwnedEntries
            let currency = ownedEntriesCurrencies |> Array.append allocationsCurrencies |> Array.distinct |> String.concat ", "

            name,
            {|
                Balance = balance
                Currency = currency
            |})

    let members = memberships |> Array.unzip |> fst |> Set

    // All transactions in the tricount:
    let entries =
        registry.AllRegistryEntry
        |> Array.map (fun x -> x.RegistryEntry)

    let parsedRows =
        entries
        |> Array.map (fun x ->
            // Members involved in the transaction:
            let allocations =
                x.Allocations
                |> Array.where (fun x -> Math.Abs(x.Amount.Value) > 0m)
                |> Array.map (fun x -> x.Membership.RegistryMembershipNonUser.Alias.DisplayName)
            // Refine allocation list to display it in a more user-friendly way:
            let allocations =
                match Set.difference members (Set allocations) |> Array.ofSeq with
                | [||] -> "all"
                | diff ->
                    let fullyExplicit = allocations |> Array.sort |> String.concat ", "
                    let formattedDiff = diff |> Array.sort |> String.concat ", "
                    let allBut = $"all except {formattedDiff}"
                    if fullyExplicit.Length < allBut.Length then fullyExplicit else allBut
            let myAllocation =
                x.Allocations
                |> Array.tryFind (fun x -> x.Membership.RegistryMembershipNonUser.Alias.DisplayName = myName)
            {
                WhoPaid = x.MembershipOwned.RegistryMembershipNonUser.Alias.DisplayName
                HowMuch = x.Amount.Value * -1m
                Currency = x.Amount.Currency
                Description = x.Description
                When = x.Date
                Involved = allocations
                HowMuchForMe = myAllocation |> Option.map (fun x -> x.Amount.Value * -1m) |> Option.defaultValue 0.00m
            })
        |> Array.sortBy (fun x -> x.When, x.Description, x.WhoPaid, x.HowMuch)

    let total =
        entries
        |> Array.where (fun x -> x.TypeTransaction <> "BALANCE") // that would be at least "NORMAL" lines, and "INCOME" lines
        |> Array.sumBy (fun x -> x.Amount.Value)
        |> Math.Abs
    let totalForMe =
        entries
        |> Array.where (fun x -> x.TypeTransaction <> "BALANCE")  // that would be at least "NORMAL" lines, and "INCOME" lines
        |> Array.choose (fun x -> x.Allocations |> Array.tryFind (fun x -> x.Membership.RegistryMembershipNonUser.Alias.DisplayName = myName))
        |> Array.sumBy (fun x -> x.Amount.Value)
        |> Math.Abs

    {
        Title = registry.Title
        Memberships = memberships
        Rows = parsedRows
        Total = total
        TotalForMe = totalForMe
    }

In [9]:
let formatTricountTable myName (tricount: ParsedTricount) =
    let currency =
        match tricount.Rows |> List.ofSeq with
        | [] -> ""
        | x :: _ -> x.Currency
    let td value = $"""<td>{value}</td>"""
    let th value = $"""<th>{value}</th>"""
    let tr (row: ResultRow) = $"""
        <tr>
            %s{td row.WhoPaid}
            %s{td $"{row.HowMuch}&nbsp;{row.Currency}"}
            %s{td row.Description}
            %s{td (row.When.Date.ToString("yyyy-MM-dd"))}
            %s{td row.Involved}
            %s{td $"{row.HowMuchForMe}&nbsp;{row.Currency}"}
        </tr>"""
    let totalRow currency total totalForMe = $"""
        <tr>
            %s{th "Total"}
            %s{th $"{total}&nbsp;{currency}"}
            %s{th ""}
            %s{th ""}
            %s{th ""}
            %s{th $"{totalForMe}&nbsp;{currency}"}
        </tr>"""
    $"""
    <html>
    <body>
    <head><style>
        html {{ font-family: Arial }}
        table {{ text-align: right }}
    </style></head>
    <h1>%s{tricount.Title}</h1>
    <table>
        <tr>
            <th>Who paid?</th>
            <th>How much?</th>
            <th>For what reasons?</th>
            <th>When?</th>
            <th>Involves?</th>
            <th>How much for {myName}?</th>
        </tr>
        %s{[for x in tricount.Rows -> tr x] |> String.concat "\n"}
        %s{totalRow currency tricount.Total tricount.TotalForMe }
    </table>
    <div>
        <h3>Balances:</h3>
        <ul>
            %s{[for name, balance in tricount.Memberships -> $"<li>{name}: {balance.Balance}&nbsp;{balance.Currency}</li>"] |> String.concat "\n"}
        </ul>
    </div>
    </body>
    </html>
    """

Now, let's execute all that and get a nicely formatted html table of our tricount:

In [10]:
let myName = "Julia"
let parsedTricount = formattedTricountJson |> parseTricountJson myName
let tricountHtmlTable = parsedTricount |> formatTricountTable myName

// display tricountHtmlTable
Microsoft.DotNet.Interactive.HtmlKernel.HTML(tricountHtmlTable)

Who paid?,How much?,For what reasons?,When?,Involves?,How much for Julia?
Julia,64.00 USD,Car,2018-10-16,all,16.00 USD
Brian,13.00 USD,Picnic,2018-10-16,all,3.25 USD
Alex,85.00 USD,Hotel,2018-10-16,all,20.00 USD
Total,162.00 USD,,,,39.25 USD
